In [1]:
import dask_distance 
import dask.array as da
import dask.dataframe as dd
import dask_ml
import time
import numpy as np
import sparse
import pandas as pd


from dask.distributed import Client
client = Client(memory_limit='12GB')

In [2]:
tmp = np.zeros(
    (25, 16)
)
tmp[2, 2] += 10.
print("Array created")
s = sparse.COO.from_numpy(tmp)
print("Array sparsified")
tmp2 = da.from_array(s, chunks=(5000, 5000))
tmp2.compute()
print(tmp2)
print("array_daskified")

Array created
Array sparsified
dask.array<array, shape=(25, 16), dtype=float64, chunksize=(25, 16), chunktype=sparse.COO>
array_daskified


In [3]:
import sparse
x = da.random.random((25076, 160901), chunks=(5000, 5000))
x[x < 0.99983] = 0
s = x.map_blocks(sparse.COO)
s

dask.array<COO, shape=(25076, 160901), dtype=float64, chunksize=(5000, 5000), chunktype=sparse.COO>

In [ ]:
x = da.random.random((n_users + n_ingredients, n_interactions), chunksize)
y = (n_interactsions) 

tr

In [ ]:
start = time.time()
n = da.matmul(s, s.T)
n.compute()
print(time.time() - start)

import numpy as np
from scipy.spatial.distance import cosine
x = np.array([
    [0, -1, 2, 0, 0],
    [-1, 0, 3, 0, 0],
    [0, 1, 0, -2, 0]
]).astype(float)

cosine(x[0], x[1])

dot_mat = x @ x.T

from math import sqrt
1 - dot_mat[0][1] / (sqrt(dot_mat[0][0]) * sqrt(dot_mat[1][1]))

In [4]:
seed = 25
ddf = dd.read_csv("data/interactions_train.csv")
train, val = dask_ml.model_selection.train_test_split(
    ddf, 
    test_size=0.1, 
    train_size=0.9,
    shuffle=True,
    random_state=seed)
print(len(train), len(val))

628841 70060


In [82]:
ddf.rating_normalized = (ddf.rating - ddf.rating.mean()) / ddf.rating.std()

In [84]:
import numpy as np

class SimilarityScorer:
    def __init__(self, interactions):
        start = time.time()
        print("Generating indices")
        user_codes, self.user_idx_to_id = pd.factorize(ddf.user_id.compute())
        recipe_codes, self.recipe_idx_to_id = pd.factorize(ddf.recipe_id.compute())
        self.user_id_to_idx = {user: idx for idx, user in enumerate(self.user_idx_to_id)}
        self.recipe_id_to_idx = {recipe: idx for idx, recipe in enumerate(self.recipe_idx_to_id)}
        
        s = sparse.COO(
            [user_codes, recipe_codes],
            ddf.rating_normalized.compute(),
            shape=(len(self.user_idx_to_id), len(self.recipe_idx_to_id)),
            fill_value=0
        )

        self.sparse_mat = da.from_array(s, chunks=(5000, 5000))
        print("Generating dot products", time.time() - start)

        self.dot_product_similarities = self.sparse_mat @ self.sparse_mat.T
        self.dot_product_similarities.compute()
        print("Done with similarities!", time.time() - start)

    def predict_topk_for_user(self, user_id, k):
        user_idx = scorer.user_id_to_idx[user_id]
        similarities = scorer.dot_product_similarities[user_idx]
        similarities_norm = similarities / similarities.sum()
        recs_raw = similarities_norm.reshape(1, -1) @ scorer.sparse_mat
        recs = recs_raw.map_blocks(lambda x: x.todense(), dtype=np.ndarray)
        rec_values = recs.topk(k).compute()
        rec_idxs = recs.argtopk(k).compute()
        recs_ids = [scorer.recipe_idx_to_id[idx] for idx in rec_idxs]
        return recs_ids, rec_values
    
    def predict_pair(self, user_id, recipe_id):
        user_idx = scorer.user_id_to_idx[user_id]
        recipe_idx = scorer.recipe_id_to_idx[recipe_id]
        similarities = scorer.dot_product_similarities[user_idx]
        similarities_norm = similarities / similarities.sum()
        predicted_score = similarities_norm @ self.sparse_mat[:, recipe_idx]
        return predicted_score.compute()

scorer = SimilarityScorer(ddf)

Generating indices
Generating dot products 1.139784812927246
Done with similarities! 5.883272886276245


In [85]:
user_id = 2002254807
scorer.predict_topk_for_user(user_id, 10)

([Int64Index([27520, 26849, 129615, 89204, 34382, 29765, 31128, 88486, 17126,
              71373],
             dtype='int64')],
 array([[1.11208334, 0.34257697, 0.20500826, 0.15315892, 0.11120465,
         0.10046562, 0.09979406, 0.09971496, 0.09500543, 0.09384096]]))

In [86]:
user_id = 2002254807
recipe_id = 27520
scorer.predict_pair(user_id, recipe_id)

1.1120833398163439

In [76]:
ddf[ddf.user_id == 2002254807].compute()

,user_id,recipe_id,date,rating,u,i
698215,2002254807,45317,2018-08-23,0.0,11624,49698
698219,2002254807,129615,2018-08-23,5.0,11624,145756
698291,2002254807,26849,2018-08-28,5.0,11624,49659
698373,2002254807,141983,2018-09-04,0.0,11624,63569
698391,2002254807,27520,2018-09-07,5.0,11624,64311


In [81]:
ddf[ddf.recipe_id == 27208].compute()

,user_id,recipe_id,date,rating,u,i
4322,39194,27208,2002-05-04,0.0,402,99787
4580,28455,27208,2002-05-13,5.0,1048,99787
5329,41850,27208,2002-06-02,5.0,13444,99787
5376,39038,27208,2002-06-02,0.0,20778,99787
5392,35140,27208,2002-06-03,5.0,2741,99787
...,...,...,...,...,...,...
688328,10951,27208,2015-12-25,5.0,8532,99787
691567,1581225,27208,2016-10-25,4.0,2587,99787
697785,2002204415,27208,2018-07-05,4.0,6734,99787
698660,2001346577,27208,2018-10-25,5.0,5202,99787
